In [329]:
import pandas as pd
import os
pwd = os.getcwd()
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import openpyxl
import json
import requests


In [362]:
df_raw = pd.read_csv(pwd + "/data/activeltcoutbreak.csv")
df_raw.columns


Index(['Report_Data_Extracted', 'PHU_Num', 'PHU', 'LTC_Home', 'LTCH_Num',
       'City', 'Beds', 'Total_LTC_Resident_Cases', 'Total_LTC_Resident_Deaths',
       'Total_LTC_HCW_Cases'],
      dtype='object')

In [363]:
# First case of COVID-19 in Ontario, Canada was reported on January 25, 2020 (https://pmc.ncbi.nlm.nih.gov/articles/PMC9922963/#:~:text=In%20Ontario%2C%20Canada%2C%20the%20first,the%20first%20lockdown%20in%20Ontario.). 
# Therefore that is the start date for this analysis. 
reference_date = pd.to_datetime('2020-01-25')

#Due to challenges finding data for the first case, the assumption is made that any LTCH without 0 cases is considered the first "case"
# Similarly the first instance where LTCH deaths is not equal to 0 is considered the first death

Case_df = (
    df_raw[df_raw['Total_LTC_Resident_Cases'] != "0"]  # filter out rows without cases
    .assign(
        Report_Data_Extracted=pd.to_datetime(df_raw['Report_Data_Extracted']), # convert to date
        First_Case_Num_Days=lambda x: (x['Report_Data_Extracted'] - reference_date).dt.days #calculate difference in days
    )
    .sort_values(by='First_Case_Num_Days')  # sort by 'First_Case_Num_Days'
    .drop_duplicates(subset='LTCH_Num', keep='first')  # drop duplicates based on 'LTCH_Num'
    .loc[:, ['LTC_Home', 'First_Case_Num_Days']]  # column selection
)
Case_df

,LTC_Home,First_Case_Num_Days
0,"Albright Gardens Homes, Incorporated",90
106,Seven Oaks,90
105,Royal Terrace,90
104,Royal Rose Place,90
100,ReachView Village,90
...,...,...
82984,Highland Wood,1036
83811,EMO Health Centre,1046
89488,West Lake Terrace,1110
90556,Broadview Nursing Centre,1138


In [364]:
Death_df = (
    df_raw[df_raw['Total_LTC_Resident_Deaths'] != "0"]  # filter out rows without deaths
    .assign(
        Report_Data_Extracted=pd.to_datetime(df_raw['Report_Data_Extracted']), # convert to date
        First_Death_Num_Days=lambda x: (x['Report_Data_Extracted'] - reference_date).dt.days #calculate difference in days
    )
    .sort_values(by='First_Death_Num_Days')  # sort by 'First_Case_Num_Days'
    .drop_duplicates(subset='LTCH_Num', keep='first')  # drop duplicates based on 'LTCH_Num'
    .loc[:, ['LTC_Home', 'First_Death_Num_Days']]  # column selection
    # .drop_duplicates(subset='LTC_Home', keep='first') #dropping duplicates

)

Q1_df = pd.merge(Case_df, Death_df, on='LTC_Home', how='left')
Q1_df


,LTC_Home,First_Case_Num_Days,First_Death_Num_Days
0,"Albright Gardens Homes, Incorporated",90,377.0
1,Seven Oaks,90,90.0
2,Royal Terrace,90,935.0
3,Royal Rose Place,90,90.0
4,ReachView Village,90,95.0
...,...,...,...
616,Highland Wood,1036,NaN
617,EMO Health Centre,1046,1059.0
618,West Lake Terrace,1110,1110.0
619,Broadview Nursing Centre,1138,NaN


In [365]:
df_paper = pd.read_excel(pwd + "/data/NH_data_.XLSX")
df_paper = df_paper.rename(columns={"LTCH": "LTC_Home"})
df_paper


,LTC_Home,PRIV_BEDS,SEMI_BEDS,THREE_BEDS,BASIC_BEDS,CONCARE_BED,RES_BED,INT_BED,ACCREDITATION,YEAR_RENO,...,Resident_death,Resident_deaths_per100beds,Had-outbreak_WAVE1,Local_incidence_WAVE1,Resident_deaths_WAVE1,Resident_deaths_per100beds_WAVE1,Had-outbreak_WAVE2,Local_incidence_WAVE2,Resident_deaths_WAVE2,Resident_deaths_per100beds_WAVE2
0,AFTON PARK PLACE LONG TERM CARE COMMUNITY,76.0,52.0,0.0,0.0,0,0,0,0,0,...,0,0.0,YES,"Medium (150-299 cases per 100,000 before Sept....",0,0.0,YES,Medium local incidence,0,0.0
1,"ALBRIGHT GARDENS HOMES, INCORPORATED",167.0,64.0,0.0,0.0,0,0,0,0,0,...,1,0.4,YES,"Medium (150-299 cases per 100,000 before Sept....",0,0.0,YES,Medium local incidence,1,0.4
2,ALEXANDER PLACE,80.0,48.0,0.0,0.0,0,2,0,1,0,...,1,0.8,YES,"Medium (150-299 cases per 100,000 before Sept....",0,0.0,YES,Medium local incidence,1,0.8
3,ALGOMA MANOR NURSING HOME,48.0,48.0,0.0,0.0,1,0,0,1,0,...,0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0,0.0,NO,Low local incidence,0,0.0
4,ALGONQUIN NURSING HOME,1.0,24.0,0.0,48.0,0,1,1,1,1,...,0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0,0.0,YES,Low local incidence,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,YEE HONG CENTRE - MISSISSAUGA,88.0,112.0,0.0,0.0,0,0,0,1,0,...,1,0.5,NO,"High (300+ cases per 100,000 before Sept. 1, 2...",0,0.0,YES,High local incidence,1,0.5
623,YEE HONG CENTRE - SCARBOROUGH FINCH,110.0,140.0,0.0,0.0,0,1,0,1,0,...,0,0.0,YES,"High (300+ cases per 100,000 before Sept. 1, 2...",0,0.0,YES,High local incidence,0,0.0
624,YEE HONG CENTRE - SCARBOROUGH MCNICOLL,67.0,88.0,0.0,0.0,0,0,0,1,0,...,0,0.0,YES,"High (300+ cases per 100,000 before Sept. 1, 2...",0,0.0,YES,High local incidence,0,0.0
625,YORK REGION MAPLE HEALTH CENTRE,68.0,32.0,0.0,0.0,15,3,0,1,0,...,1,1.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0,0.0,YES,High local incidence,1,1.0


In [366]:

# Standardize the LTC_Home columns
Q1_df['LTC_Home'] = Q1_df['LTC_Home'].str.strip().str.lower()
df_paper['LTC_Home'] = df_paper['LTC_Home'].str.strip().str.lower()

#Merging
Q2_df = pd.merge(Q1_df, df_paper, on='LTC_Home', how='left')
Q2_df


,LTC_Home,First_Case_Num_Days,First_Death_Num_Days,PRIV_BEDS,SEMI_BEDS,THREE_BEDS,BASIC_BEDS,CONCARE_BED,RES_BED,INT_BED,...,Resident_death,Resident_deaths_per100beds,Had-outbreak_WAVE1,Local_incidence_WAVE1,Resident_deaths_WAVE1,Resident_deaths_per100beds_WAVE1,Had-outbreak_WAVE2,Local_incidence_WAVE2,Resident_deaths_WAVE2,Resident_deaths_per100beds_WAVE2
0,"albright gardens homes, incorporated",90,377.0,167.0000,64.000000,0.000000,0.000000,0.0,0.0,0.0,...,1.0,0.4,YES,"Medium (150-299 cases per 100,000 before Sept....",0.0,0.0,YES,Medium local incidence,1.0,0.4
1,seven oaks,90,90.0,137.0000,112.000000,0.000000,0.000000,17.0,2.0,0.0,...,41.0,16.5,YES,"High (300+ cases per 100,000 before Sept. 1, 2...",41.0,16.5,YES,High local incidence,0.0,0.0
2,royal terrace,90,935.0,61.8663,50.106115,0.000000,14.008993,0.0,0.0,0.0,...,0.0,0.0,YES,"Medium (150-299 cases per 100,000 before Sept....",0.0,0.0,YES,Medium local incidence,0.0,0.0
3,royal rose place,90,90.0,58.0000,38.000000,0.000000,0.000000,0.0,0.0,0.0,...,20.0,20.8,YES,"Medium (150-299 cases per 100,000 before Sept....",20.0,20.8,YES,Medium local incidence,0.0,0.0
4,reachview village,90,95.0,3.0000,24.000000,3.000000,72.000000,0.0,0.0,0.0,...,17.0,17.0,YES,"Medium (150-299 cases per 100,000 before Sept....",17.0,17.0,YES,Medium local incidence,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,highland wood,1036,NaN,18.0000,12.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0.0,0.0,NO,Low local incidence,0.0,0.0
617,emo health centre,1046,1059.0,61.8663,50.106115,1.657468,14.008993,0.0,0.0,0.0,...,0.0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0.0,0.0,NO,Low local incidence,0.0,0.0
618,west lake terrace,1110,1110.0,7.0000,12.000000,0.000000,28.000000,0.0,0.0,0.0,...,0.0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0.0,0.0,NO,Low local incidence,0.0,0.0
619,broadview nursing centre,1138,NaN,5.0000,26.000000,0.000000,44.000000,0.0,0.0,0.0,...,0.0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0.0,0.0,NO,Low local incidence,0.0,0.0


In [375]:
Q2_missing = Q2_df.isna().sum() / len(Q2_df) * 100 
# <4% missingness in all columns upon merging the second dataset, I will leave it as is
print(Q2_missing.to_string())


LTC_Home                             0.000000
First_Case_Num_Days                  0.000000
First_Death_Num_Days                15.297907
PRIV_BEDS                            3.864734
SEMI_BEDS                            3.864734
THREE_BEDS                           3.864734
BASIC_BEDS                           3.864734
CONCARE_BED                          3.864734
RES_BED                              3.864734
INT_BED                              3.864734
ACCREDITATION                        3.864734
YEAR_RENO                            3.864734
CON_Y                                3.864734
Total_Beds                           3.864734
PER_FEM_LTCR                         3.864734
PER_LTCR<65                          3.864734
PER_LTCR>85                          3.864734
PER_LTCR_DEMENTIA                    3.864734
PER_LTCR_CHF                         3.864734
PR_ANTIP_MED                         3.864734
PR_PREUL                             3.864734
PR_FELL                           

In [368]:

duplicates = Q2_df[Q2_df.duplicated(keep=False)] 
duplicates # no duplicates shown

,LTC_Home,First_Case_Num_Days,First_Death_Num_Days,PRIV_BEDS,SEMI_BEDS,THREE_BEDS,BASIC_BEDS,CONCARE_BED,RES_BED,INT_BED,...,Resident_death,Resident_deaths_per100beds,Had-outbreak_WAVE1,Local_incidence_WAVE1,Resident_deaths_WAVE1,Resident_deaths_per100beds_WAVE1,Had-outbreak_WAVE2,Local_incidence_WAVE2,Resident_deaths_WAVE2,Resident_deaths_per100beds_WAVE2


In [369]:
# # Select only the numerical columns for centering and standardizing
# numerical_cols = Q2_df.select_dtypes(include=['float64', 'int64'])

# # Centering and standardizing: subtract the mean and divide by the standard deviation
# Q2_df_centered_standardized = (numerical_cols - numerical_cols.mean()) / numerical_cols.std()

# # Replace the original numerical columns with the centered and standardized versions
# Q2_df[numerical_cols.columns] = Q2_df_centered_standardized

# Q2_df

In [370]:
# Specify the columns to center and standardize
cols_to_center = ['First_Case_Num_Days', 'First_Death_Num_Days']

 # Centering and standardizing: subtract the mean and divide by the standard deviation
Q2_df[cols_to_center] = Q2_df[cols_to_center].apply(lambda x: (x - x.mean()) / x.std())

Q2_df


,LTC_Home,First_Case_Num_Days,First_Death_Num_Days,PRIV_BEDS,SEMI_BEDS,THREE_BEDS,BASIC_BEDS,CONCARE_BED,RES_BED,INT_BED,...,Resident_death,Resident_deaths_per100beds,Had-outbreak_WAVE1,Local_incidence_WAVE1,Resident_deaths_WAVE1,Resident_deaths_per100beds_WAVE1,Had-outbreak_WAVE2,Local_incidence_WAVE2,Resident_deaths_WAVE2,Resident_deaths_per100beds_WAVE2
0,"albright gardens homes, incorporated",-1.127265,-0.514579,167.0000,64.000000,0.000000,0.000000,0.0,0.0,0.0,...,1.0,0.4,YES,"Medium (150-299 cases per 100,000 before Sept....",0.0,0.0,YES,Medium local incidence,1.0,0.4
1,seven oaks,-1.127265,-1.364071,137.0000,112.000000,0.000000,0.000000,17.0,2.0,0.0,...,41.0,16.5,YES,"High (300+ cases per 100,000 before Sept. 1, 2...",41.0,16.5,YES,High local incidence,0.0,0.0
2,royal terrace,-1.127265,1.137048,61.8663,50.106115,0.000000,14.008993,0.0,0.0,0.0,...,0.0,0.0,YES,"Medium (150-299 cases per 100,000 before Sept....",0.0,0.0,YES,Medium local incidence,0.0,0.0
3,royal rose place,-1.127265,-1.364071,58.0000,38.000000,0.000000,0.000000,0.0,0.0,0.0,...,20.0,20.8,YES,"Medium (150-299 cases per 100,000 before Sept....",20.0,20.8,YES,Medium local incidence,0.0,0.0
4,reachview village,-1.127265,-1.349272,3.0000,24.000000,3.000000,72.000000,0.0,0.0,0.0,...,17.0,17.0,YES,"Medium (150-299 cases per 100,000 before Sept....",17.0,17.0,YES,Medium local incidence,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,highland wood,1.934716,NaN,18.0000,12.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0.0,0.0,NO,Low local incidence,0.0,0.0
617,emo health centre,1.967084,1.504076,61.8663,50.106115,1.657468,14.008993,0.0,0.0,0.0,...,0.0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0.0,0.0,NO,Low local incidence,0.0,0.0
618,west lake terrace,2.174237,1.655031,7.0000,12.000000,0.000000,28.000000,0.0,0.0,0.0,...,0.0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0.0,0.0,NO,Low local incidence,0.0,0.0
619,broadview nursing centre,2.264866,NaN,5.0000,26.000000,0.000000,44.000000,0.0,0.0,0.0,...,0.0,0.0,NO,"Low (<150 cases per 100,000 before Sept. 1, 2020)",0.0,0.0,NO,Low local incidence,0.0,0.0


In [377]:
Q2_df["First_Case_Num_Days"]

0     -1.127265
1     -1.127265
2     -1.127265
3     -1.127265
4     -1.127265
         ...   
616    1.934716
617    1.967084
618    2.174237
619    2.264866
620    2.316654
Name: First_Case_Num_Days, Length: 621, dtype: float64

In [387]:
#Q3
x = Q2_df.drop(["LTC_Home", "First_Case_Num_Days", "First_Death_Num_Days"], axis=1)
# Keep only numerical columns
x = x.select_dtypes(include=['float64', 'int64'])
x = x.fillna(x.mean()) #simply replace with mean

y = Q2_df["First_Case_Num_Days"]
x.head()

,PRIV_BEDS,SEMI_BEDS,THREE_BEDS,BASIC_BEDS,CONCARE_BED,RES_BED,INT_BED,ACCREDITATION,YEAR_RENO,CON_Y,...,B_Beds,C_Beds,D_Upgrade_Beds,ELDCAP_Beds,Resident_death,Resident_deaths_per100beds,Resident_deaths_WAVE1,Resident_deaths_per100beds_WAVE1,Resident_deaths_WAVE2,Resident_deaths_per100beds_WAVE2
0,167.0000,64.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,19.0,...,0.0,0.0,0.0,0.0,1.0,0.4,0.0,0.0,1.0,0.4
1,137.0000,112.000000,0.0,0.000000,17.0,2.0,0.0,1.0,0.0,32.0,...,0.0,249.0,0.0,0.0,41.0,16.5,41.0,16.5,0.0,0.0
2,61.8663,50.106115,0.0,14.008993,0.0,0.0,0.0,0.0,0.0,32.0,...,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,58.0000,38.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,20.0,20.8,20.0,20.8,0.0,0.0
4,3.0000,24.000000,3.0,72.000000,0.0,0.0,0.0,1.0,1.0,48.0,...,0.0,100.0,0.0,0.0,17.0,17.0,17.0,17.0,0.0,0.0


In [389]:

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor


x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
model = LinearRegression()
model.fit(x_train, y_train)

vif_data = pd.DataFrame()
vif_data["feature"] = x.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(x.values, i) 
                   for i in range(len(x.columns))]

print(vif_data)

                             feature           VIF
0                          PRIV_BEDS  1.430214e+01
1                          SEMI_BEDS  9.401140e+00
2                         THREE_BEDS  1.332407e+00
3                         BASIC_BEDS  3.429509e+00
4                        CONCARE_BED  1.365695e+00
5                            RES_BED  1.580460e+00
6                            INT_BED  1.325399e+00
7                      ACCREDITATION  4.777347e+00
8                          YEAR_RENO  1.753320e+00
9                              CON_Y  6.903233e+00
10                        Total_Beds  5.407074e+02
11                      PER_FEM_LTCR  1.057716e+02
12                       PER_LTCR<65  7.534259e+00
13                       PER_LTCR>85  6.728344e+01
14                 PER_LTCR_DEMENTIA  5.346475e+01
15                      PER_LTCR_CHF  9.635674e+00
16                      PR_ANTIP_MED  7.984724e+00
17                          PR_PREUL  5.801713e+00
18                           PR

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
